# Riproduzione modelli e addestramento

In [1]:
import pandas as pd 
import numpy as np
import os
dataset_folder = 'dataset_new'

In [2]:
sensors = ['sensor2', 'sensor3', 'sensor4','sensor6','sensor7','sensor8','sensor9','sensor11','sensor12','sensor13','sensor14',
           'sensor15','sensor17','sensor20', 'sensor21']

In [3]:
train = []
files = os.listdir(dataset_folder)
train_files = [file for file in files if file.startswith('scaled')]

for file in files: 
    path_to_file = dataset_folder + '/'+file
    df = pd.read_csv(path_to_file, index_col=0) #index_col = 0 perché sennò viene aggiunta la colonna unnamed. 
    train.append(df)

train[0].head() 


,unit_number,time_cycle,op_setting1,op_setting2,op_setting3,RUL,sensor2,sensor3,sensor4,sensor6,...,sensor8,sensor9,sensor11,sensor12,sensor13,sensor14,sensor15,sensor17,sensor20,sensor21
0,1.0,1.0,-0.0005,0.0004,100.0,258.0,0.355972,0.370523,0.308580,1.0,...,0.623529,0.204233,0.348571,0.231279,0.642857,0.239116,0.647755,0.272727,0.559524,0.446331
1,1.0,2.0,0.0008,-0.0003,100.0,257.0,0.388759,0.399100,0.309360,1.0,...,0.647059,0.202431,0.308571,0.236882,0.654762,0.278567,0.685659,0.363636,0.488095,0.534836
2,1.0,3.0,-0.0014,-0.0002,100.0,256.0,0.313817,0.353298,0.445398,1.0,...,0.664706,0.241484,0.302857,0.217015,0.636905,0.264526,0.564462,0.272727,0.404762,0.458577
3,1.0,4.0,-0.0020,0.0001,100.0,255.0,0.487119,0.417107,0.237285,1.0,...,0.647059,0.215326,0.314286,0.240448,0.684524,0.245612,0.558909,0.363636,0.470238,0.391966
4,1.0,5.0,0.0016,0.0000,100.0,254.0,0.196721,0.476218,0.321217,1.0,...,0.670588,0.268799,0.262857,0.245033,0.654762,0.252109,0.556736,0.363636,0.577381,0.543371


Proviamo, al contrario di quanto fatto in precedenza, ad addestrare le reti in maniera separata sulle 4 componenti del dataframe e poi confrontare i dati 

In [4]:
##Preparo i dati di FD0001 per l'input. 
sequence_length = 30 #Rappresenta i Time Steps del layer di input

from keras.utils import pad_sequences

columns_to_keep = sensors 
columns_to_keep.append('RUL')
df = train[0]
selected_columns = df[sensors]
dataset_array = selected_columns.values



sequences = [dataset_array[i:i+sequence_length] for i in range(len(dataset_array) - sequence_length + 1)]

sequences_padded = pad_sequences(sequences, maxlen=sequence_length, dtype='float32', padding='post')

X = sequences_padded[:, :, :-1]
y = sequences_padded[:, :, -1]


2023-09-17 12:11:35.091003: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-17 12:11:36.433028: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/extras/CUPTI/lib64:/home/aliquodfahriam/miniconda3/envs/tensorflowEnv/lib/
2023-09-17 12:11:36.433369: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/

In [5]:
print(X.shape, y.shape)

(24691, 30, 15) (24691, 30)


In [6]:
#Splitting del dataset
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
from keras import backend as K 
import tensorflow as tf 
import math 
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 


# def custom_score(y_true, y_pred):
#      sum = 0 
#      for true, predicted in zip(y_true, y_pred): 
#           d_i = predicted - true
#           if d_i < 0:
#              esponente = 1/(d_i/13)
#              sum += (math.e ** esponente) -1 
#           else: 
#              esponente =  d_i/10
#              sum += (math.e ** esponente) -1 
     
#      return sum 

@tf.function
def custom_score2(y_true, y_pred):
    d_i = y_pred - y_true
    esponente = tf.where(d_i < 0, 1.0 / (d_i / 13.0), d_i / 10.0)
    sum = tf.reduce_sum(tf.exp(esponente) - 1.0)
    
    return sum


In [8]:
from keras import regularizers
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Flatten

#Replica della rete neurale proposta come "small LSTM"
def createLSTMsmallModel(l1Nodes, l2Nodes, d1Nodes, d2Nodes, input_shape):
    # input layer
    lstm1 = LSTM(l1Nodes, input_shape=input_shape, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))
    
    lstm2 = LSTM(l2Nodes, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))    
    flatten = Flatten()
    dense1 = Dense(d1Nodes, activation='relu', kernel_regularizer=regularizers.l2(0.1))
    dense2 = Dense(d2Nodes, activation='relu', kernel_regularizer=regularizers.l2(0.1))
    
    # output layer
    outL = Dense(1, activation='relu', kernel_regularizer=regularizers.l2(0.1))
    
    #layers
    layers = [lstm1, lstm2,flatten,  dense1, dense2, outL]
    
    model = Sequential(layers)
    #Abbiamo aggiunto le nostre funzioni di loss e accuracy definite precedentemente
    model.compile(optimizer='adam', loss=root_mean_squared_error, metrics = ['accuracy', custom_score2] )
    return model


def createLSTMlargeModel(l1Nodes, l2Nodes, l3Nodes, l4Nodes, d1dense , d2dense, input_shape): 
    lstm1 = LSTM(l1Nodes, input_shape=input_shape, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))
    d1 = Dropout(0.2)
    lstm2 = LSTM(l2Nodes, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))
    d2 = Dropout(0.2)
    lstm3 = LSTM(l3Nodes, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))
    d3 = Dropout(0.2)
    lstm4 = LSTM(l4Nodes, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))
    d4 = Dropout(0.2)

    flatten = Flatten()

    dense1 = Dense(d1dense,activation='relu', kernel_regularizer=regularizers.l2(0.1))
    dense2 = Dense(d2dense,activation='relu', kernel_regularizer=regularizers.l2(0.1))
    output_dense = Dense(1,activation='relu', kernel_regularizer=regularizers.l2(0.1))

    layers = [lstm1,d1, lstm2,d2, lstm3, d3, lstm4, d4,flatten,  dense1, dense2, output_dense]

    model = Sequential(layers)
    model.compile(optimizer='adam', loss=root_mean_squared_error, metrics = ['accuracy', custom_score2])
    return model


In [9]:
LSTMlargeModel= createLSTMlargeModel(128, 64, 32, 16, 64, 32, (30,15))
LSTMsmallModel = createLSTMsmallModel(60, 30, 30, 15, (30, 15)) 

2023-09-17 12:11:37.918525: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-17 12:11:37.960782: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-17 12:11:37.960976: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-17 12:11:37.961928: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [10]:
LSTMsmallModel.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 30, 60)            18240     
                                                                 
 lstm_5 (LSTM)               (None, 30, 30)            10920     
                                                                 
 flatten_1 (Flatten)         (None, 900)               0         
                                                                 
 dense_3 (Dense)             (None, 30)                27030     
                                                                 
 dense_4 (Dense)             (None, 15)                465       
                                                                 
 dense_5 (Dense)             (None, 1)                 16        
                                                                 
Total params: 56,671
Trainable params: 56,671
Non-trai

In [11]:
LSTMlargeModel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 128)           73728     
                                                                 
 dropout (Dropout)           (None, 30, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 30, 64)            49408     
                                                                 
 dropout_1 (Dropout)         (None, 30, 64)            0         
                                                                 
 lstm_2 (LSTM)               (None, 30, 32)            12416     
                                                                 
 dropout_2 (Dropout)         (None, 30, 32)            0         
                                                                 
 lstm_3 (LSTM)               (None, 30, 16)            3

In [12]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model, save_model

EPOCHS = 100

path_small = './models/LSTMsmall/FD0001'
path_large = './models/LSTMlarge/FD0001'


model_checkpoint = ModelCheckpoint(
    path_small,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',

)


early_stopping = EarlyStopping(
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
)



In [13]:
history_small = LSTMsmallModel.fit(
    X_train, 
    y_train, 
    validation_data=(X_val, y_val), 
    epochs = EPOCHS,
    batch_size = 32,
    use_multiprocessing =True, 
    callbacks=[model_checkpoint, early_stopping])

Epoch 1/100


2023-09-17 12:11:43.280276: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100


614/618 [============================>.] - ETA: 0s - loss: 100.3836 - accuracy: 0.0041 - custom_score2: 811592960.0000
Epoch 1: val_loss improved from inf to 76.89383, saving model to ./models/LSTMsmall/FD0001


INFO:tensorflow:Assets written to: ./models/LSTMsmall/FD0001/assets


INFO:tensorflow:Assets written to: ./models/LSTMsmall/FD0001/assets


...

INFO:tensorflow:Assets written to: ./models/LSTMsmall/FD0001/assets


618/618 [==============================] - 10s 17ms/step - loss: 61.0693 - accuracy: 0.0041 - custom_score2: 8450003173376.0000 - val_loss: 59.4700 - val_accuracy: 0.0038 - val_custom_score2: 1320587755520.0000
Epoch 24/100
615/618 [============================>.] - ETA: 0s - loss: 61.0144 - accuracy: 0.0041 - custom_score2: 7282702155776.0000
Epoch 24: val_loss did not improve from 59.47000
618/618 [==============================] - 5s 9ms/step - loss: 61.0300 - accuracy: 0.0041 - custom_score2: 7247359901696.0000 - val_loss: 59.6923 - val_accuracy: 0.0038 - val_custom_score2: 890724417536.0000
Epoch 25/100
615/618 [============================>.] - ETA: 0s - loss: 60.6049 - accuracy: 0.0041 - custom_score2: 4794484260864.0000
Epoch 25: val_loss improved from 59.47000 to 59.31364, saving model to ./models/LSTMsmall/FD0001


INFO:tensorflow:Assets written to: ./models/LSTMsmall/FD0001/assets


INFO:tensorflow:Assets written to: ./models/LSTMsmall/FD0001/assets


618/618 [==============================] - 10s 17ms/step - loss: 60.6039 - accuracy: 0.0041 - custom_score2: 4778115465216.0000 - val_loss: 59.3136 - val_accuracy: 0.0038 - val_custom_score2: 10472216592384.0000
Epoch 26/100
618/618 [==============================] - ETA: 0s - loss: 60.7493 - accuracy: 0.0041 - custom_score2: 16417036435456.0000
Epoch 26: val_loss did not improve from 59.31364
618/618 [==============================] - 6s 9ms/step - loss: 60.7493 - accuracy: 0.0041 - custom_score2: 16417036435456.0000 - val_loss: 59.4276 - val_accuracy: 0.0038 - val_custom_score2: 390722781184.0000
Epoch 27/100
616/618 [============================>.] - ETA: 0s - loss: 60.3426 - accuracy: 0.0041 - custom_score2: 9689763938304.0000
Epoch 27: val_loss did not improve from 59.31364
618/618 [==============================] - 6s 9ms/step - loss: 60.3385 - accuracy: 0.0041 - custom_score2: 9659503083520.0000 - val_loss: 63.0013 - val_accuracy: 0.0038 - val_custom_score2: 454023380992.0000
Ep

In [14]:
model_checkpoint = ModelCheckpoint(
    path_large,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',

)
history_large = LSTMlargeModel.fit(
    X_train, 
    y_train, 
    validation_data=(X_val, y_val), 
    epochs = EPOCHS,
    batch_size = 32,
    use_multiprocessing =True, 
    callbacks=[model_checkpoint, early_stopping])

Epoch 1/100
618/618 [==============================] - ETA: 0s - loss: 97.3901 - accuracy: 0.0041 - custom_score2: 42135188.0000
Epoch 1: val_loss improved from inf to 80.09718, saving model to ./models/LSTMlarge/FD0001


INFO:tensorflow:Assets written to: ./models/LSTMlarge/FD0001/assets


INFO:tensorflow:Assets written to: ./models/LSTMlarge/FD0001/assets


618/618 [==============================] - 27s 35ms/step - loss: 97.3901 - accuracy: 0.0041 - custom_score2: 42135188.0000 - val_loss: 80.0972 - val_accuracy: 0.0038 - val_custom_score2: 20657280.0000
Epoch 2/100
617/618 [============================>.] - ETA: 0s - loss: 76.0969 - accuracy: 0.0041 - custom_score2: 11533552640.0000
Epoch 2: val_loss improved from 80.09718 to 68.77796, saving model to ./models/LSTMlarge/FD0001


INFO:tensorflow:Assets written to: ./models/LSTMlarge/FD0001/assets


...

INFO:tensorflow:Assets written to: ./models/LSTMlarge/FD0001/assets


618/618 [==============================] - 23s 37ms/step - loss: 62.7331 - accuracy: 0.0041 - custom_score2: 10799680585728.0000 - val_loss: 59.9627 - val_accuracy: 0.0038 - val_custom_score2: 427940839424.0000
Epoch 15/100
617/618 [============================>.] - ETA: 0s - loss: 62.0774 - accuracy: 0.0041 - custom_score2: 28466560892928.0000
Epoch 15: val_loss did not improve from 59.96266
618/618 [==============================] - 10s 17ms/step - loss: 62.0888 - accuracy: 0.0041 - custom_score2: 28420499046400.0000 - val_loss: 60.2443 - val_accuracy: 0.0038 - val_custom_score2: 1295386279936.0000
Epoch 16/100
618/618 [==============================] - ETA: 0s - loss: 61.9971 - accuracy: 0.0041 - custom_score2: 4948674740224.0000
Epoch 16: val_loss did not improve from 59.96266
618/618 [==============================] - 10s 17ms/step - loss: 61.9971 - accuracy: 0.0041 - custom_score2: 4948674740224.0000 - val_loss: 61.2848 - val_accuracy: 0.0038 - val_custom_score2: 1010029428736.00

INFO:tensorflow:Assets written to: ./models/LSTMlarge/FD0001/assets


INFO:tensorflow:Assets written to: ./models/LSTMlarge/FD0001/assets


618/618 [==============================] - 24s 39ms/step - loss: 61.5608 - accuracy: 0.0041 - custom_score2: 53027335569408.0000 - val_loss: 59.1160 - val_accuracy: 0.0038 - val_custom_score2: 338953732096.0000
Epoch 19/100
618/618 [==============================] - ETA: 0s - loss: 61.7338 - accuracy: 0.0041 - custom_score2: 18021735202816.0000
Epoch 19: val_loss did not improve from 59.11596
618/618 [==============================] - 11s 18ms/step - loss: 61.7338 - accuracy: 0.0041 - custom_score2: 18021735202816.0000 - val_loss: 59.8374 - val_accuracy: 0.0038 - val_custom_score2: 1096471478272.0000
Epoch 20/100
617/618 [============================>.] - ETA: 0s - loss: 61.2569 - accuracy: 0.0041 - custom_score2: 37449925394432.0000
Epoch 20: val_loss did not improve from 59.11596
618/618 [==============================] - 13s 21ms/step - loss: 61.2628 - accuracy: 0.0041 - custom_score2: 37389326090240.0000 - val_loss: 60.1921 - val_accuracy: 0.0038 - val_custom_score2: 221236527104.0